<a href="https://colab.research.google.com/github/Edmond292000/Recognition/blob/main/DawnBench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making the Most of your Colab Subscription



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
import matplotlib.pyplot as plt

torch.backends.cudnn.benchmark = True
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=4)

# hyper parameters
num_epochs = 200
learning_rate = 0.001
channels = 32
depth = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # shortcut
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = self.shortcut(x)
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity
        out = F.relu(out)
        return out


class Cifar10(nn.Module):
    def __init__(self, num_classes):
        super(Cifar10, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)

        self.layer1 = self._make_layer(32, 32, 2, stride=1)
        self.layer2 = self._make_layer(32, 64, 2, stride=2)
        self.layer3 = self._make_layer(64, 128, 2, stride=2)
        self.layer4 = self._make_layer(128, 256, 2, stride=2)
        self.layer5 = self._make_layer(256, 512, 2, stride=2)

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, in_channels, out_channels, blocks, stride):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_channels, out_channels, 1))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))

        x = self.layer1(x)
        #x = F.dropout2d(x, 0.2, training=self.training)

        x = self.layer2(x)
        #x = F.dropout2d(x, 0.25, training=self.training)

        x = self.layer3(x)
        #x = F.dropout2d(x, 0.3, training=self.training)

        x = self.layer4(x)
        #x = F.dropout2d(x, 0.3, training=self.training)

        x = self.layer5(x)
        # x = F.dropout2d(x, 0.25, training=self.training)

        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)
        #x = F.dropout(x, 0.6, training=self.training)
        x = self.fc(x)
        return x


num_classes = 10
model = Cifar10(num_classes)
model = model.to(device)
print(next(model.parameters()).device)
# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0007, weight_decay=1e-4)
# cheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 150], gamma=0.1)

criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

losses = []
train_accuracy_list = []
test_accuracy_list = []

start_time = time.time()
for epoch in range(num_epochs):
    epoch_start_time = time.time()
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for img, label in trainloader:
        img, label = img.to(device), label.to(device)

        output = model(img)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = output.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()

    train_accuracy = 100.0 * correct / total
    avg_loss = running_loss / len(trainloader)
    losses.append(avg_loss)
    train_accuracy_list.append(train_accuracy)

    epoch_end_time = time.time() - epoch_start_time
    print(
        f"Epoch [{epoch + 1}/{num_epochs}], "f"Loss: {running_loss / len(trainloader):.4f}, "f"Train Accuracy: {train_accuracy:.2f}%",
        f"Time: {epoch_end_time}: ")

    # ---Testing---
    model.eval()
    with torch.no_grad():
        correct_test = 0
        total_test = 0
        for img, label in testloader:
            img, label = img.to(device), label.to(device)
            output = model(img)
            _, predicted = output.max(1)
            total_test += label.size(0)
            correct_test += predicted.eq(label).sum().item()

        test_accuracy = 100.0 * correct_test / total_test
        test_accuracy_list.append(test_accuracy)

        print(f"Test Accuracy: {test_accuracy:.2f}%")
        if test_accuracy >= 93.0:
            torch.save(model.state_dict(), "best_model.pth")
            print(f"Achieving 94% in {time.time() - start_time:.2f} seconds")

total_training_time = time.time() - start_time
print(f"Total {total_training_time / 60} minutes")

plt.figure(figsize=(8, 5))
plt.plot(losses, label="Train Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.legend()
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(train_accuracy_list, label="Train Accuracy")
plt.plot(test_accuracy_list, label="Test Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Train vs Test Accuracy")
plt.legend()
plt.show()


100%|██████████| 170M/170M [00:14<00:00, 11.6MB/s]


cuda:0
Epoch [1/200], Loss: 1.4686, Train Accuracy: 45.83% Time: 8.601047277450562: 
Test Accuracy: 56.03%
Epoch [2/200], Loss: 1.0049, Train Accuracy: 64.05% Time: 6.3609161376953125: 
Test Accuracy: 65.76%
Epoch [3/200], Loss: 0.7874, Train Accuracy: 72.22% Time: 6.722705364227295: 
Test Accuracy: 76.25%
Epoch [4/200], Loss: 0.6721, Train Accuracy: 76.39% Time: 6.9357452392578125: 
Test Accuracy: 74.97%
Epoch [5/200], Loss: 0.5920, Train Accuracy: 79.50% Time: 6.736664056777954: 
Test Accuracy: 78.22%
Epoch [6/200], Loss: 0.5392, Train Accuracy: 81.41% Time: 6.504906415939331: 
Test Accuracy: 80.12%
Epoch [7/200], Loss: 0.4938, Train Accuracy: 82.88% Time: 6.596717834472656: 
Test Accuracy: 78.89%
Epoch [8/200], Loss: 0.4582, Train Accuracy: 84.26% Time: 6.504001140594482: 
Test Accuracy: 82.77%
Epoch [9/200], Loss: 0.4352, Train Accuracy: 85.11% Time: 6.572409629821777: 
Test Accuracy: 83.34%
Epoch [10/200], Loss: 0.4046, Train Accuracy: 86.00% Time: 6.4492528438568115: 
Test Accura